# Extract the dataset

In [2]:
import zipfile
import os
import shutil

def extract_images(path):
    for root, subdir, files in os.walk(img_path):
        for f in files:
            if '.jpg' in f:
                return False
    return True


base_path = os.getcwd()
img_path = f"{base_path}/data/intel_image_classification"

if extract_images(img_path):
    with zipfile.ZipFile(f"{base_path}/data/archive.zip", 'r') as zip_ref:
        zip_ref.extractall(img_path)
        # We are only interested on model performance, so no prediction data is used
        shutil.rmtree(f"{img_path}/seg_pred")

# Load images

In [10]:
from tensorflow import keras
import numpy as np
from sklearn import preprocessing
from tensorflow.keras.preprocessing import image

def prepare_targets(y_train, y_test):
	le = preprocessing.LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

def import_data(path):
    x_train_list, y_train_list, x_test_list, y_test_list = [], [], [], []

    for root, subdir, files in os.walk(img_path):
        if '/seg_train/seg_train/' in root:
            for f in files:
                file_path = os.path.normpath(os.path.join(root, f))
                label = file_path.split(os.sep)[len(file_path.split(os.sep)) - 2]
                x_train_list.append(
                    image.img_to_array(
                        image.load_img(
                            file_path, 
                            target_size=(224, 224)
                            )
                        )
                    )
                y_train_list.append(label)
        if '/seg_test/seg_test/' in root:
            for f in files:
                file_path = os.path.normpath(os.path.join(root, f))
                label = file_path.split(os.sep)[len(file_path.split(os.sep)) - 2]
                x_test_list.append(
                    image.img_to_array(
                        image.load_img(
                            file_path, 
                            target_size=(224, 224)
                            )
                        )
                    )
                y_test_list.append(label)

    y_train, y_test = prepare_targets(
        np.array(y_train_list),
        np.array(y_test_list)
        )

    return (np.concatenate(x_train_list, axis=0), y_train), (np.concatenate(x_test_list, axis=0), y_test)

In [9]:

(x_train, y_train), (x_test, y_test) = import_data(img_path)

ValueError: zero-dimensional arrays cannot be concatenated